**1) Common mistakes to avoid:**

- ❌ Wrapping **objects in [ ]**.
- ❌ Using **quotes for numbers**.
- ❌ Using **"true"** instead of **true**.

**2) Important points about NULLs in JSON & PySpark**

- Use `null (not "null")`.
- Spark automatically converts `null → NULL`.

| Wrong                     | Correct                 |
| ------------------------- | ----------------------- |
| `{"name":}`               | `{"name":null} or {"name":""} |
| `"Zipcode":"null"`        | `"Zipcode":null`        |
| `"Decommisioned":"false"` | `"Decommisioned":false` |

##### 3) Quick Debug Trick

In [0]:
df = spark.read \
    .option("mode","PERMISSIVE") \
    .option("columnNameOfCorruptRecord","_corrupt_record") \
    .json(path)

df.filter("_corrupt_record is not null").show(truncate=False)

##### 4) Invalid JSON Syntax

**a) Trailing comma**

     # Error in Spark: _corrupt_record
     {"id":1,"name":"A",}

**b) Missing quotes for strings**

     {"id":1,"name":A}

**c) Single quotes instead of double quotes**

     # JSON requires double quotes
     {'id':1,'name':'A'}

##### 5) Mixing JSON Objects and Arrays Incorrectly

**a) Array but read without multiLine=true**

      [
        {"id":1},
        {"id":2}
      ]

- **Spark issue**: All rows become **NULL**.

- **Fix**

      spark.read.option("multiLine","true").json(path)

##### 6) One JSON object per file vs One per line (Most common!)

Multiple JSON objects without newline

      {"id":1}{"id":2}

**Spark:** _corrupt_record

**Correct**

      {"id":1}
      {"id":2}

##### 7) Inconsistent Schema Across Files

**File1**

      {"id":1,"price":100}

**File2**

      {"id":"ABC","price":"cheap"}

**Spark issue:**
- NULL values
- Wrong data types

**Fix**

      # Provide schema:
      spark.read.schema(schema).json(path)

##### 8) Comments in JSON (Not Allowed!)

      {
        "id":1, // user id
        "name":"A"
      }

- **Spark: Fails to parse**

##### 9) Special Characters Not Escaped

     ❌
     {"text":"Hello "Spark""}

     ✅
     {"text":"Hello \"Spark\""}